In [14]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm
import os
import yaml
import random
import operator

In [3]:
def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))

In [4]:
#Question 1
ratings = {'1-star':0, '2-star':0, '3-star':0, '4-star':0, '5-star':0}
for d in data:
    ratings[str(int(d['review/taste']))+'-star'] += 1
print('Rounded down ratings: {}'.format(ratings))

Rounded down ratings: {'1-star': 554, '2-star': 2723, '3-star': 12934, '4-star': 29458, '5-star': 4331}


In [5]:
#Question 2
listOfBeers = set()
ratings = dict()
averageRatings = dict()
for d in data:
    listOfBeers.add(d['beer/name'])
for beerName in listOfBeers:
    ratings[beerName] = 0
    averageRatings[beerName] = 0
for d in data:
    ratings[d['beer/name']] += 1
    averageRatings[d['beer/name']] += d['review/overall']
for key in ratings.keys():
    averageRatings[key] = -1 if ratings[key] < 5 else averageRatings[key]/ratings[key]
maximum = max(averageRatings.values())
result = []
for item in averageRatings.items():
    if item[1] == maximum:
        result.append(item)
print(result)

[('Wobbly Bob APA', 4.714285714285714)]


In [6]:
#Question 3
data2 = [d for d in data if 'beer/ABV' in d.keys() and 'beer/style' in d.keys()]
def feature(datum):
    feat = [1]
    if datum['beer/style'] == 'Hefeweizen':
        feat.append(1)
    else:
        feat.append(0)
    feat.append(datum['beer/ABV'])
    return feat
X = [feature(d) for d in data2]
y = [d['review/taste'] for d in data2]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y, rcond=None)
print(theta)
#theta[0] shows us the offset. Theta[1] is the weighting of the beer being a 
#Hefeweizen on the taste score, in this case it has negative impact on the score.
#Theta[2] shows the impact of the beer alcohol content on the taste score, in this case
#higher is better

[ 3.11795084 -0.05637406  0.10877902]


In [7]:
#Question 4
train = data2[:len(data2)//2]
X_train = [feature(d) for d in train]
y_train = [d['review/taste'] for d in train]
theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train, rcond=-1)
predictions = numpy.matmul(theta, numpy.transpose(X_train))
print('Error for train data =', numpy.sum(numpy.square(numpy.subtract(predictions, y_train))))

test = data2[len(data2)//2:]
X_test = [feature(d) for d in test]
y_test = [d['review/taste'] for d in test]
predictions = numpy.matmul(theta, numpy.transpose(X_test))
print('Error for test data =', numpy.sum(numpy.square(numpy.subtract(predictions, y_test))))


Error for train data = 12099.201400335607
Error for test data = 10592.663029965459


In [8]:
#Question 5
dataCopy = data2.copy()
for i in range(10):
    random.shuffle(dataCopy)

train = dataCopy[:len(dataCopy)//2]
X_train = [feature(d) for d in train]
y_train = [d['review/taste'] for d in train]
theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train, rcond=-1)
predictions = numpy.matmul(theta, numpy.transpose(X_train))
print('Error for train data =', numpy.sum(numpy.square(numpy.subtract(predictions, y_train))))

test = dataCopy[len(dataCopy)//2:]
X_test = [feature(d) for d in test]
y_test = [d['review/taste'] for d in test]
predictions = numpy.matmul(theta, numpy.transpose(X_test))
print('Error for test data =', numpy.sum(numpy.square(numpy.subtract(predictions, y_test))))

#The data that is pulled from the website might be in some certain order and dividing it before shuffling
#might result in skewed data

Error for train data = 11137.399495328096
Error for test data = 11348.452104890428


In [15]:
#Question 6
def feature(datum):
    #feat = [1]
    feat = []
    feat.append(datum['review/taste'])
    feat.append(datum['review/appearance'])
    feat.append(datum['review/aroma'])
    feat.append(datum['review/palate'])
    feat.append(datum['review/overall'])
    return feat

dataCopy = data2.copy()
for i in range(10):
    random.shuffle(dataCopy)

train = dataCopy[:len(dataCopy)//2]
X_train = [feature(d) for d in train]
y_train = [1 if d['beer/style'] == 'Hefeweizen' else 0 for d in train]
clf = svm.SVC(C=1000, kernel='rbf', gamma='scale',verbose=True)
clf.fit(X_train, y_train)
train_predictions = clf.predict(X_train)
print('Error for train data =', 100*numpy.sum(numpy.square(numpy.subtract(train_predictions, y_train)))/len(y_train), '%')

test = dataCopy[len(dataCopy)//2:]
X_test = [feature(d) for d in test]
y_test = [1 if d['beer/style'] == 'Hefeweizen' else 0 for d in test]
test_predictions = clf.predict(X_test)
print('Error for test data =', 100*numpy.sum(numpy.square(numpy.subtract(test_predictions, y_test)))/len(y_test), '%')

[LibSVM]Error for train data = 1.16 %
Error for test data = 1.368 %


In [16]:
#Question 7
def feature(datum):
    feat = [1]
    #feat = []
    #feat.append(datum['review/taste'])
#     feat.append(datum['review/appearance'])
#     feat.append(datum['review/aroma'])
#     feat.append(datum['review/palate'])
#     feat.append(datum['review/overall'])
    feat.append(datum['beer/ABV'])
    return feat

train = dataCopy[:len(dataCopy)//2]
X_train = [feature(d) for d in train]
y_train = [1 if d['beer/style'] == 'Hefeweizen' else 0 for d in train]
clf2 = svm.SVC(C=1000, kernel='rbf', gamma='scale')
clf2.fit(X_train, y_train)
train_predictions = clf2.predict(X_train)
print('Error for train data =', 100*numpy.sum(numpy.square(numpy.subtract(train_predictions, y_train)))/len(y_train), '%')

test = dataCopy[len(dataCopy)//2:]
X_test = [feature(d) for d in test]
y_test = [1 if d['beer/style'] == 'Hefeweizen' else 0 for d in test]
test_predictions = clf2.predict(X_test)
print('Error for test data =', 100*numpy.sum(numpy.square(numpy.subtract(test_predictions, y_test)))/len(y_test), '%')

Error for train data = 1.2 %
Error for test data = 1.272 %


In [17]:
#Question 8
c = [0.1, 10, 1000, 100000]

for reg in c:
    clf = svm.SVC(C=reg, kernel='rbf', gamma='scale')
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_train)
    print('--------For regularization factor =', reg, '-----------')
    print('Error for train data =', 100*numpy.sum(numpy.square(numpy.subtract(train_predictions, y_train)))/len(y_train), '%')
    
    test_predictions = clf.predict(X_test)
    print('Error for test data =', 100*numpy.sum(numpy.square(numpy.subtract(test_predictions, y_test)))/len(y_test), '%')


--------For regularization factor = 0.1 -----------
Error for train data = 1.2 %
Error for test data = 1.272 %
--------For regularization factor = 10 -----------
Error for train data = 1.2 %
Error for test data = 1.272 %
--------For regularization factor = 1000 -----------
Error for train data = 1.2 %
Error for test data = 1.272 %
--------For regularization factor = 100000 -----------
Error for train data = 1.2 %
Error for test data = 1.272 %
